## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from sklearn.metrics import pairwise_distances

### Carregando o dataset

In [2]:
df = pd.read_csv('abalone.csv')
df.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Verificando as classes

In [3]:
df["Sex"].unique()

array(['M', 'F', 'I'], dtype=object)

### Verificando o nome exato das colunas

In [4]:
df.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight', 'Rings'],
      dtype='object')

### Transformando classes em números

In [5]:
df = df.rename(columns={'Sex': 'Class'})
df["Class"].replace({"M": 0, "F": 1, "I": 2}, inplace=True)
df.head()

,Class,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,2,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Verificando a corretude das classes

In [7]:
df["Class"].unique()

array([0, 1, 2], dtype=int64)

### Armazenando as classes em uma variável separada

In [8]:
labels = df["Class"].values
labels

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

### Isolando os dados da classe

In [9]:
df.drop("Class", axis=1, inplace=True)
df.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


### Transformando os dados em uma array

In [10]:
data = df.to_numpy()
data

array([[ 0.455 ,  0.365 ,  0.095 , ...,  0.101 ,  0.15  , 15.    ],
       [ 0.35  ,  0.265 ,  0.09  , ...,  0.0485,  0.07  ,  7.    ],
       [ 0.53  ,  0.42  ,  0.135 , ...,  0.1415,  0.21  ,  9.    ],
       ...,
       [ 0.6   ,  0.475 ,  0.205 , ...,  0.2875,  0.308 ,  9.    ],
       [ 0.625 ,  0.485 ,  0.15  , ...,  0.261 ,  0.296 , 10.    ],
       [ 0.71  ,  0.555 ,  0.195 , ...,  0.3765,  0.495 , 12.    ]])

## Clustering

### Inicialização da matriz de pertinência

A matriz de pertinência é inicializada aleatoriamente $u_{ik}(i=1,...c$ e $k=1,...,n)$ do objeto $k$ pertencente ao grupo $C_i$ tal que:
- $u_{ik} \in [0,1]$;
- $0 < \sum_{k=1}^nu_{ik} < n$;
- $\sum_{i=1}^cu_{ik}=1$ para todo $k \in \Omega$.

In [11]:
def init_membership_matrix(n, k):
    membership_matrix = np.random.rand(n, k) # gera uma matriz inicial aleatória com valores entre 0 e 1
    membership_matrix = membership_matrix / membership_matrix.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return membership_matrix

### Inicialização dos medoides

#### 1. Primeiro Medoide
Selecione o primeiro medoide $m_1$ como o ponto com a menor distância total para todos os outros pontos no conjunto de dados $X$, com $n$ amostras:


$m_1 = \arg \min_i \left( \sum_{j=1}^{n} d(x_i, x_j) \right)$


onde $d(x_i, x_j)$ representa a dissimilaridade entre os pontos $x_i$ e $x_j$.

#### 2. Próximos Medoides
Para cada próximo medoide $m_k$, com $k = 2, \dots, c$, encontre o ponto $x$ que maximize a menor distância em relação aos medoides já selecionados. Para cada ponto candidato $x$ (ainda não selecionado como medoide), calcule:


$\text{dist\_mínima}(x) = \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j)$


Então, selecione o ponto $x$ com a maior distância mínima como o próximo medoide:


$m_k = \arg \max_{x \in X \setminus \{m_1, \dots, m_{k-1}\}} \left( \min_{m_j \in \{m_1, \dots, m_{k-1}\}} d(x, m_j) \right)$

In [12]:
def init_medoids(X, c):
    total_distances = np.sum(pairwise_distances(X), axis=1) # distância somada de cada ponto para os outros
    
    first_medoid_idx = np.argmin(total_distances) # ponto com menor distância total
    medoids_indices = [first_medoid_idx]  # lista para os índices dos medoides
    medoids = [X[first_medoid_idx]]  # lista para os medoides

    for _ in range(1, c): # (1, c) pois já temos um medoide
        max_min_dist = -np.inf # armazena a maior entre as menores distâncias
        next_medoid_idx = -1 # armazena o índice do candidato a medoide

        for i in range(len(X)):
            if i in medoids_indices: # ignora os pontos que já são medoides
                continue 

            # primeiro ele calcula a distância do ponto para cada um dos medoides
            # depois ele seleciona a menor dessas distâncias
            min_dist = np.min([pairwise_distances(X[i].reshape(1, -1), np.array([medoid])).flatten()[0] for medoid in medoids])

            if min_dist > max_min_dist: # a maior distância entre as menores
                max_min_dist = min_dist
                next_medoid_idx = i

        # adiciona o ponto escolhido como medoide
        medoids_indices.append(next_medoid_idx)
        medoids.append(X[next_medoid_idx])

    return np.array(medoids)

### Atualização da matriz de pertinência

Fixo o protótipo, os graus de pertinência são atualizados com base nessa equação:

#### $u_{ik} = [\sum_{l=1}^c(\frac{d(x_k,v_i)}{d(x_k,v_l)})^{\frac{1}{m-1}}]^{-1}$

In [13]:
def update_membership_matrix(data, medoids, m):
    distance_matrix = pairwise_distances(data, medoids, metric='euclidean') ** 2
    distance_matrix = np.fmax(distance_matrix, np.finfo(np.float64).eps)  # evita que matriz_distancias seja 0, np.finfo... é o menor número maior que zero aqui
    
    inverse_distance_matrix = 1 / distance_matrix
    power = 1 / (m - 1)
    updated_membership_matrix = (inverse_distance_matrix ** power) / np.sum(inverse_distance_matrix ** power, axis=1, keepdims=True) # fórmula para atualizar os graus de pertinência
    
    return updated_membership_matrix

### Atualização dos medoides

Fixo os graus de pertinência, os centroides são atualizados com base nessa equação:

#### $q = \arg \min_{1 \leq j \leq n} \sum_{k=1}^{n} \left( u_{ik} \right)^m \cdot d(x_j, x_k)$

Essa fórmula busca, para cada medoide $m_i$, o ponto $p \in C_i$ que minimiza a soma das distâncias dentro do cluster, garantindo que o novo medoide minimize o custo de distância.

In [14]:
def update_medoids(X, medoids, membership_matrix, m=2):
    n, c = X.shape[0], len(medoids)
    updated_medoids = np.copy(medoids)  # cópia dos medoides 
    chosen_medoids_indices = []  # armazena os índices dos novos medoides
    distances = pairwise_distances(X, X)  # matriz de distância entre os pontos

    for i in range(c): 
        initial_medoid_idx = np.where((X == medoids[i]).all(axis=1))[0][0]  # índice do medoide atual
        min_weighted_distance = np.sum([(membership_matrix[k, i] ** m) * distances[k, initial_medoid_idx] for k in range(n)]) # distância ponderada do medoide atual
        best_medoid = medoids[i] # melhor candidato a medoide
        best_medoid_idx = initial_medoid_idx # índice do melhor candidato a medoide

        for j in range(n):
            if j in chosen_medoids_indices:  # ignorar pontos que já são medoides
                continue
            
            weighted_distance = np.sum([(membership_matrix[k, i] ** m) * distances[k, j] for k in range(n)]) # distância ponderada do candidato a medoide

            if weighted_distance < min_weighted_distance: # caso a distância ponderada do candidato seja menor que a do medoide atual
                min_weighted_distance = weighted_distance # atualiza a menor distância ponderada
                best_medoid = X[j] # atualiza o melhor candidato a medoide
                best_medoid_idx = j # atualiza o índice do melhor candidato a medoide
                updated_medoids[i] = best_medoid # já coloca ele como medoide

        updated_medoids[i] = best_medoid # atualiza de fato os medoides
        chosen_medoids_indices.append(best_medoid_idx) # para garantir que ele não será escolhido como medoide novamente

    return updated_medoids

### Fuzzy C-Medoids (FCMdd)

Etapas:
- Inicialização da matriz de pertinência
- Inicialização dos medoides
- Atualização da matriz de pertinência
- Atualização dos medoides

Critérios de parada:
- Convergência (entre os medoides)
- Número máximo de iterações

In [15]:
def fcmdd(data, k, m=2, max_iter=1000000):
    n = data.shape[0]
    membership_matrix = init_membership_matrix(n, k)
    medoids = init_medoids(data, k)
    for _ in range(max_iter):
        membership_matrix = update_membership_matrix(data, medoids, m)
        new_medoids = update_medoids(data, medoids, membership_matrix, m)
        if np.array_equal(medoids, new_medoids): # se os medoides não mudaram, para
            break
        medoids = new_medoids
    return medoids, membership_matrix

### Índice de Rand Ajustado (IRA)

In [16]:
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)

### Simulação de Monte Carlo

In [17]:
def monte_carlo_fuzzy_simulation(X, true_labels, k, m=2, num_trials=100):
    results = []
    for trial in range(num_trials):
        print(trial)
        medoids, membership_matrix = fcmdd(X, k, m)
        predicted_labels = np.argmax(membership_matrix, axis=1)
        rand_idx = indice_rand(true_labels, predicted_labels)
        results.append(rand_idx)
    
    mean_ari = np.mean(results)
    std_ari = np.std(results)
    return mean_ari, std_ari

### Definição de parâmetros e execução do método

In [18]:
k = 3
num_trials = 100
m = 2
mean_rand_index, std_rand_index = monte_carlo_fuzzy_simulation(data, labels, k, m, num_trials)

print(f"Monte Carlo FCMdd Clustering Results ({num_trials} trials)")
print(f"Mean Rand Index: {mean_rand_index:.4f}")
print(f"Standard Deviation of Rand Index: {std_rand_index:.4f}")

0


KeyboardInterrupt: 